In [1]:
#scenario 1
import numpy as np
import pyDOE
import random
import os
import sys
import matplotlib.pyplot as plt
import flopy
import pandas as pd


def generate_initial_lhs(n, d, bounds):
    lhs = pyDOE.lhs(d, samples=n)
    X_init = np.zeros((n, d))
    for i in range(d):
        X_init[:, i] = bounds[i][0] + (bounds[i][1] - bounds[i][0]) * lhs[:, i]
    return X_init



def initialize_annealing_parameters():
    T0 = 1000
    alpha = 0.95
    Nmax = 1000
    return T0, alpha, Nmax



def cost_function(X):
    n = X.shape[0]
    min_dist = float('inf')
    for i in range(n):
        for j in range(i + 1, n):
            dist = np.linalg.norm(X[i] - X[j])
            min_dist = min(min_dist, dist)
    return min_dist



def generate_neighbor(X, mode='both'):
    n, d = X.shape
    X_candidate = X.copy()

    if mode == 'Q_only':

        swap_indices = random.sample(range(n), max(1, n // 10))  # 扰动10%的样本
        np.random.shuffle(X_candidate[swap_indices, 0])

    elif mode == 'R_only':

        swap_indices = random.sample(range(n), max(1, n // 10))
        np.random.shuffle(X_candidate[swap_indices, 0])

    elif mode == 'both':

        j = random.choice([0, 1])
        swap_indices = random.sample(range(n), max(1, n // 10))
        X_candidate[swap_indices, j] = np.random.permutation(X_candidate[swap_indices, j])

    return X_candidate



def metropolis_acceptance(DeltaC, T):
    if DeltaC > 0:
        return True
    else:
        return random.random() < np.exp(DeltaC / T)



def should_terminate(improvement_rates, T, T_min, Nmax, iteration):
    if len(improvement_rates) >= 10 and all(abs(rate) < 0.01 for rate in improvement_rates[-10:]):
        return True
    if T <= T_min:
        return True
    if iteration >= Nmax:
        return True
    return False



def simulated_annealing(X_init, T0, alpha, Nmax, T_min=1, mode='both'):
    X_current = X_init
    C_current = cost_function(X_current)
    improvement_rates = []

    for iteration in range(Nmax):
        X_candidate = generate_neighbor(X_current, mode=mode)
        C_candidate = cost_function(X_candidate)
        DeltaC = C_candidate - C_current

        if metropolis_acceptance(DeltaC, T0):
            X_current = X_candidate
            C_current = C_candidate

        improvement_rate = (C_candidate - C_current) / C_current if C_current != 0 else 0
        improvement_rates.append(improvement_rate)

        if should_terminate(improvement_rates, T0, T_min, Nmax, iteration):
            break

        T0 = alpha * T0

    return X_current


output_dir = "sample"
os.makedirs(output_dir, exist_ok=True) 


sim_name = "Sub"
length_units = "meters"
time_units = "days"


nper = 30   
nlay = 1
nrow = 30  
ncol = 50  
delr = 100.0
delc = 100.0
top = 100.0  
botm = np.full((nlay, nrow, ncol), 60.0) 
icelltype = 1
strt = np.full((nlay, nrow, ncol), 70.0) 


chd_spd = {
    0: [
        [0, row, 6, 100] for row in range(0, 2)
    ] + [
        [0, 2, col , 100] for col in range(4, 7)
    ] + [
        [0, row, 4, 100] for row in range(2, 5)
    ] + [
        [0, 6, col, 100] for col in range(2, 4)
    ] + [
        [0, row , 4, 100] for row in range(4, 7)
    ] + [
        [0, 8, col, 100] for col in range(0, 2)
    ] + [
        [0, row, 2, 100] for row in range(6, 9)
    ] + [
        [0, row, 41, 80] for row in range(28, 30)
    ] + [
        [0, 28, col, 80] for col in range(42, 44)
    ] + [
        [0, row , 43, 80] for row in range(26, 29)
    ] + [
        [0, 26, col, 80] for col in range(44, 46)
    ] + [
        [0, row, 45, 80] for row in range(24, 27)
    ] + [
        [0, 24, col, 80] for col in range(46, 48)
    ] + [
        [0, row, 47, 80] for row in range(22, 25)
    ] + [
        [0, 22, col, 80] for col in range(48, 50)
    ] 
}
unique_chd_spd = {0: []}
used_cells = set()
for item in chd_spd[0]:
    lay, row, col, _ = item
    cell_key = (lay, row, col)
    if cell_key not in used_cells:
        unique_chd_spd[0].append(item)
        used_cells.add(cell_key)
    else:
        print(f"已跳过重复的定水头单元格: ({lay}, {row}, {col})")

chd_spd = unique_chd_spd


idomain = np.ones((nlay, nrow, ncol), dtype=np.int32)


original_regions = [
    (0, 0, *range(0, 6)),
    (0, 2, *range(0, 4)),
    (0, 4, *range(0, 4)),
    (0, 6, *range(0, 2)),
    (0, 8, 0),
    (0, 12, *range(0, 2)),
    (0, 14, *range(0, 2)),
    (0, 16, *range(0, 4)),
    (0, 18, *range(0, 6)),
    (0, 20, *range(0, 8)),
    (0, 22, *range(0, 24)),
    (0, 24, *range(0, 26)),
    (0, 26, *range(0, 32)),
    (0, 28, *range(0, 34)),
    (0, 29, *range(0, 38))
]


connected_regions = []


for i in range(len(original_regions)):
    if i == 0:

        connected_regions.append(original_regions[i])
    else:

        prev_row = original_regions[i - 1][1]
        current_row = original_regions[i][1]
        prev_cols = set(original_regions[i - 1][2:])
        current_cols = set(original_regions[i][2:])


        min_col = min(min(prev_cols), min(current_cols))
        max_col = max(max(prev_cols), max(current_cols))

        for row in range(prev_row + 1, current_row):
            connected_regions.append((0, row, *range(min_col, max_col + 1)))


        connected_regions.append(original_regions[i])


for region in connected_regions:
    lay = region[0]
    row = region[1]
    for col in region[2:]:
        idomain[lay, row, col] = -1


for row in range(8, 10):
    idomain[0, row, :2] = 1

original_regions = [
    (0, 0, *range(16, 50)),
    (0, 2, *range(20, 50)),
    (0, 4, *range(30, 50)),
    (0, 6, *range(32, 50)),
    (0, 8, *range(42, 50)),
    (0, 10, *range(46, 50)),
    (0, 12, *range(48, 50)),
    (0, 24, *range(48, 50)),
    (0, 26, *range(46, 50)),
    (0, 28, *range(44, 50)),
    (0, 29, *range(42, 50)),
]


connected_regions = []

for i in range(len(original_regions)):
    if i == 0:

        connected_regions.append(original_regions[i])
    else:

        prev_row = original_regions[i - 1][1]
        current_row = original_regions[i][1]
        prev_cols = set(original_regions[i - 1][2:])
        current_cols = set(original_regions[i][2:])


        min_col = min(min(prev_cols), min(current_cols))
        max_col = max(max(prev_cols), max(current_cols))


        for row in range(prev_row + 1, current_row):
            connected_regions.append((0, row, *range(min_col, max_col + 1)))

        connected_regions.append(original_regions[i])


for region in connected_regions:
    lay = region[0]
    row = region[1]
    for col in region[2:]:
        idomain[lay, row, col] = -1


for row in range(14, 23):
    idomain[0, row, 48:] = 1
    
laytyp = 1  


hk_original = np.ones((nlay, nrow, ncol), dtype=np.float32)


hk_original[:, 0, 3:8] = 0.0004 * 864
hk_original[:, 1, 2:10] = 0.0004 * 864
hk_original[:, 2, 2:9] = 0.0004 * 864
hk_original[:, 3, 1:8] = 0.0004 * 864
hk_original[:, 4, 0:7] = 0.0004 * 864
hk_original[:, 5, 1:6] = 0.0004 * 864
hk_original[:, 6, 1:5] = 0.0004 * 864
hk_original[:, 7, 2:4] = 0.0004 * 864

hk_original[:, 2, 9:15] = 0.0002 * 864
hk_original[:, 3, 8:16] = 0.0002 * 864
hk_original[:, 4, 7:21] = 0.0002 * 864
hk_original[:, 5, 7:23] = 0.0002 * 864
hk_original[:, 6, 10:23] = 0.0002 * 864
hk_original[:, 7, 14:21] = 0.0002 * 864
hk_original[:, 8, 16:19] = 0.0002 * 864

hk_original[:, 5, 6] = 0.0001 * 864
hk_original[:, 6, 5:10] = 0.0001 * 864
hk_original[:, 7, 4:12] = 0.0001 * 864
hk_original[:, 8, 3:11] = 0.0001 * 864
hk_original[:, 9, 4:10] = 0.0001 * 864

hk_original[:, 7, 12:14] = 0.0003 * 864
hk_original[:, 8, 11:16] = 0.0003 * 864
hk_original[:, 9, 10:17] = 0.0003 * 864
hk_original[:, 10, 12:18] = 0.0003 * 864
hk_original[:, 11, 13:19] = 0.0003 * 864
hk_original[:, 12, 16:19] = 0.0003 * 864
hk_original[:, 13, 17:20] = 0.0003 * 864
hk_original[:, 14, 19] = 0.0003 * 864

hk_original[:, 6, 23] = 0.0007 * 864
hk_original[:, 7, 21:25] = 0.0007 * 864
hk_original[:, 8, 19:25] = 0.0007 * 864
hk_original[:, 9, 16:25] = 0.0007 * 864
hk_original[:, 10, 17:25] = 0.0007 * 864
hk_original[:, 11, 18:24] = 0.0007 * 864
hk_original[:, 12, 18:23] = 0.0007 * 864
hk_original[:, 13, 19:22] = 0.0007 * 864
hk_original[:, 14, 20] = 0.0007 * 864


hk = np.ones((nlay, nrow, ncol), dtype=np.float32)


row_scale = 2
col_scale = 2


for lay in range(nlay):
    for row in range(15):
        for col in range(25):
            new_row = row * row_scale
            new_col = col * col_scale
            hk[lay, new_row:new_row + row_scale, new_col:new_col + col_scale] = hk_original[lay, row, col]

num_stress_periods = 30  

rbot = np.linspace(90., 80.25, num=nrow)

riv_spd = {}
for sp in range(num_stress_periods):
    rstage = np.linspace(90.1, 81.25, num=nrow)
    riv_spd[sp] = []
    for col in [14]:
        for row in range(2, 11):      
            s = rstage[row - 2]
            b = rbot[row - 2]  
            riv_spd[sp].append([0, row, col, s, 50, b])



perlen = 6 * 30  
nstp = 1  
tsmult = 1  
sy = np.full((nlay, nrow, ncol), 0.2)  
ss = np.full((nlay, nrow, ncol), 1e-5)




permeability_values = [0.0004 * 864, 0.0002 * 864, 0.0001 * 864, 0.0003 * 864, 0.0007 * 864]


recharge = np.zeros((nlay, nrow, ncol))


evaporation = np.zeros((nlay, nrow, ncol))


rch_spd = {}
evt_spd = {}
for sp in range(nper):
    if sp % 2 == 1:  
       
        recharge_factors = {
            0.0004 * 864: np.random.uniform(0.016 * 1.1 / 100, 0.016 * 1.3 / 100),
            0.0002 * 864: np.random.uniform(0.012 * 1.1 / 100, 0.012 * 1.3 / 100),
            0.0001 * 864: np.random.uniform(0.008 * 1.1 / 100, 0.008 * 1.3 / 100),
            0.0003 * 864: np.random.uniform(0.014 * 1.1 / 100, 0.014 * 1.3 / 100),
            0.0007 * 864: np.random.uniform(0.018 * 1.1 / 100, 0.018 * 1.3 / 100)
        }
       
        evaporation_factors = {
            0.0004 * 864: np.random.uniform(0.002 * 0.7 / 100, 0.002 * 0.9 / 100),
            0.0002 * 864: np.random.uniform(0.003 * 0.7 / 100, 0.003 * 0.9 / 100),
            0.0001 * 864: np.random.uniform(0.001 * 0.7 / 100, 0.001 * 0.9 / 100),
            0.0003 * 864: np.random.uniform(0.004 * 0.7 / 100, 0.004 * 0.9 / 100),
            0.0007 * 864: np.random.uniform(0.005 * 0.7 / 100, 0.005 * 0.9 / 100)
        }
    else:  
       
        recharge_factors = {
            0.0004 * 864: np.random.uniform(0.016 * 0.7 / 100, 0.016 * 0.9 / 100),
            0.0002 * 864: np.random.uniform(0.012 * 0.7 / 100, 0.012 * 0.9 / 100),
            0.0001 * 864: np.random.uniform(0.008 * 0.7 / 100, 0.008 * 0.9 / 100),
            0.0003 * 864: np.random.uniform(0.014 * 0.7 / 100, 0.014 * 0.9 / 100),
            0.0007 * 864: np.random.uniform(0.018 * 0.7 / 100, 0.018 * 0.9 / 100)
        }

        evaporation_factors = {
            0.0004 * 864: np.random.uniform(0.002 * 1.1 / 100, 0.002 * 1.3 / 100),
            0.0002 * 864: np.random.uniform(0.003 * 1.1 / 100, 0.003 * 1.3 / 100),
            0.0001 * 864: np.random.uniform(0.001 * 1.1 / 100, 0.001 * 1.3 / 100),
            0.0003 * 864: np.random.uniform(0.004 * 1.1 / 100, 0.004 * 1.3 / 100),
            0.0007 * 864: np.random.uniform(0.005 * 1.1 / 100, 0.005 * 1.3 / 100)
        }

    current_rch = recharge.copy()
    current_evt = evaporation.copy()

    for k in permeability_values:
        rch_mask = np.isclose(hk, k)
        current_rch[rch_mask] = recharge_factors[k]

        evt_mask = np.isclose(hk, k)
        current_evt[evt_mask] = evaporation_factors[k]

    rch_spd[sp] = current_rch
    evt_spd[sp] = []
    for lay in range(nlay):
        for row in range(nrow):
            for col in range(ncol):
                if current_evt[lay, row, col] > 0:
                    lay = int(lay)
                    row = int(row)
                    col = int(col)
                    rate = float(current_evt[lay, row, col])
                    # 假设 surface 为 100.0（可根据实际情况修改）
                    surface = 100.0

                    depth = 5.0
                    evt_spd[sp].append([lay, row, col, surface, rate, depth])


# Solver parameters
nouter, ninner = 100, 300
hclose, rclose, relax = 1e-6, 1e-6, 1.0

if __name__ == "__main__":
    n_Q = 4 
    d_Q = 10  
    bounds_Q = [(50, 100)] * d_Q  

    n_R_initial = 1  
    d_Ri = 20  
    bounds_Ri = [(90.10, 92.50)] * d_Ri  

    n_R_final = 1  
    d_Rf = 20  
    bounds_Rf = [(81.25, 82.50)] * d_Rf  
    for run in range(100):
 
        X_init_Q = generate_initial_lhs(n_Q, d_Q, bounds_Q)
        X_init_R_initial = generate_initial_lhs(n_R_initial, d_Ri, bounds_Ri)
        X_init_R_final = generate_initial_lhs(n_R_final, d_Rf, bounds_Rf)


        T0, alpha, Nmax = initialize_annealing_parameters()


        X_Q_optimized = simulated_annealing(X_init_Q, T0, alpha, Nmax, mode='Q_only')
        Q_optimized = X_Q_optimized


        X_R_initial_optimized = simulated_annealing(X_init_R_initial, T0, alpha, Nmax, mode='R_only')
        initial_water_level = X_R_initial_optimized


        X_R_final_optimized = simulated_annealing(X_init_R_final, T0, alpha, Nmax, mode='R_only')
        final_water_level = X_R_final_optimized


        for i in range(initial_water_level.shape[1]):
            if initial_water_level[0, i] < final_water_level[0, i]:
                initial_water_level[0, i], final_water_level[0, i] = final_water_level[0, i], initial_water_level[0, i]

        output_path = os.path.join(output_dir, f"optimized_Q_values_{run}.txt")
        np.savetxt(output_path, Q_optimized)
        output_path = os.path.join(output_dir, f"optimized_R_initial_values_{run}.txt")
        np.savetxt(output_path, initial_water_level)
        output_path = os.path.join(output_dir, f"optimized_R_final_values_{run}.txt")
        np.savetxt(output_path, final_water_level)


        n_wells = 10

        d_well = 3
        bounds_well = [(0, nlay - 1), (0, nrow - 1), (0, ncol - 1)]
 
        X_init_well = generate_initial_lhs(n_wells, d_well, bounds_well)

        X_init_well = np.round(X_init_well).astype(int)

        valid_well_locations = []
        for location in X_init_well:
            lay, row, col = location
            if idomain[lay, row, col] == 1:
                valid_well_locations.append((lay, row, col))
        while len(valid_well_locations) < n_wells:
            lay = np.random.randint(0, nlay)
            row = np.random.randint(0, nrow)
            col = np.random.randint(0, ncol)
            if idomain[lay, row, col] == 1:
                location = (lay, row, col)
                if location not in valid_well_locations:
                    valid_well_locations.append(location)

        X_init_well = np.array(valid_well_locations)


        T0, alpha, Nmax = initialize_annealing_parameters()


        X_well_optimized = simulated_annealing(X_init_well, T0, alpha, Nmax, mode='both')

        X_well_optimized = np.round(X_well_optimized).astype(int)
        well_locations = []
        for location in X_well_optimized:
            lay, row, col = location
            if idomain[lay, row, col] == 1:
                well_locations.append((lay, row, col))
        while len(well_locations) < n_wells:
            lay = np.random.randint(0, nlay)
            row = np.random.randint(0, nrow)
            col = np.random.randint(0, ncol)
            if idomain[lay, row, col] == 1:
                location = (lay, row, col)
                if location not in well_locations:
                    well_locations.append(location)

        c_values = np.array([0.5371, 0.9508, 0.8966, 0.9340, 
                0.6388 , 0.5806, 0.6135, 0.6934, 0.9771, 0.8470 ])
        c_values = c_values*20
        wel_spd = {
            i: [
                [lay, row, col, 0, 0] for lay, row, col in well_locations
            ] for i in range(4)
        }


        for stress_period in range(min(4, len(Q_optimized))):
            q_values = Q_optimized[stress_period]
            for i, well in enumerate(wel_spd[stress_period]):
                well[3] = q_values[i]
                well[4] = c_values[i]

        well_locations_array = np.array(well_locations)
        output_path_wells = os.path.join(output_dir, f"well_locations_{run}.txt")
        np.savetxt(output_path_wells, well_locations_array)
        

        sim_ws = os.path.join('conc_models', f'{sim_name}_{run}') 
        sim = flopy.mf6.MFSimulation(sim_name=f'{sim_name}_{run}', sim_ws=sim_ws, exe_name='../modflow/mf6.6.1_linux/bin/mf6') 
         6 个月
        tdis_ds = [(perlen, nstp, tsmult) for _ in range(nper)] 
        flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_ds, time_units=time_units) 


        gwfname =  f"gwf_{sim_name}_{run}"
        gwf = flopy.mf6.ModflowGwf(sim, modelname=gwfname, newtonoptions="NEWTON UNDER_RELAXATION", 
                                   save_flows=True) 

        imsgwf = flopy.mf6.ModflowIms(
            sim,
            print_option="SUMMARY",
            outer_dvclose=hclose,
            outer_maximum=nouter,
            under_relaxation="NONE",
            inner_maximum=ninner,
            inner_dvclose=hclose,
            rcloserecord=rclose,
            linear_acceleration="BICGSTAB",
            scaling_method="NONE",
            reordering_method="NONE",
            relaxation_factor=relax,
            filename="{}.ims".format(gwfname),
          )
        sim.register_ims_package(imsgwf, [gwf.name])

        flopy.mf6.ModflowGwfdis(gwf, length_units=length_units, nlay=nlay, nrow=nrow,
                                ncol=ncol, delr=delr, delc=delc, top=top, botm=botm, idomain=idomain)  

        flopy.mf6.ModflowGwfnpf(gwf, icelltype=icelltype, k=hk)  

        flopy.mf6.ModflowGwfic(gwf, strt=strt)  。

        flopy.mf6.ModflowGwfriv(gwf, stress_period_data=riv_spd, pname="RIV-1", print_flows=True, save_flows=True)  # 定义河流边界条件。
        flopy.mf6.ModflowGwfsto(gwf, sy=sy,ss=ss, transient={0: True})
        flopy.mf6.ModflowGwfwel(gwf, stress_period_data=wel_spd, auxiliary=['CONCENTRATION'], 
                                pname="WEL-1", save_flows=True)  

        flopy.mf6.ModflowGwfrcha(gwf, recharge=rch_spd)  
        flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_spd)  
        flopy.mf6.ModflowGwfevt(gwf, stress_period_data=evt_spd, pname="EVT-1", print_flows=True, save_flows=True)

        
        headfile = f"{sim_name}_{run}.hds"
        head_filerecord = [headfile]
        budgetfile = f"{sim_name}_{run}.bud"
        budget_filerecord = [budgetfile]
        saverecord = [("HEAD", "ALL"), ("BUDGET", "ALL")]
        flopy.mf6.ModflowGwfoc(gwf, saverecord=saverecord, head_filerecord=head_filerecord,
                               budget_filerecord=budget_filerecord, printrecord=saverecord)




        gwtname = f"gwt_{sim_name}_{run}"
        gwt = flopy.mf6.ModflowGwt(sim, modelname=gwtname, model_nam_file=f"{gwtname}.nam")


        imsgwt = flopy.mf6.ModflowIms(
            sim,
            print_option="SUMMARY",
            outer_dvclose=hclose,
            outer_maximum=nouter,
            under_relaxation="NONE",
            inner_maximum=ninner,
            inner_dvclose=hclose,
            rcloserecord=rclose,
            linear_acceleration="BICGSTAB",
            scaling_method="NONE",
            reordering_method="NONE",
            relaxation_factor=relax,
            filename="{}.ims".format(gwtname),
        )
        sim.register_ims_package(imsgwt, [gwt.name])


        flopy.mf6.ModflowGwtdis(gwt, nlay=nlay, nrow=nrow, ncol=ncol,
                                delr=delr, delc=delc, top=top, botm=botm, idomain=idomain)


        initial_concentration = 0.0  
        flopy.mf6.ModflowGwtic(gwt, strt=initial_concentration)


        dsp = flopy.mf6.ModflowGwtdsp(gwt,
                            alh=40.0,  
                            ath1=4.0,  
                            atv=0.1,   
                            diffc=0.0) 
   
        mst = flopy.mf6.ModflowGwtmst(gwt, porosity=0.3)  
        sourcerecarray = [('WEL-1', 'AUX', 'CONCENTRATION')]
        
        flopy.mf6.ModflowGwtssm(
              gwt,
              sources = sourcerecarray,
              print_flows=True,
              filename=f"{gwtname}.ssm"
         )


    
        gwt_obs_package_name = f"gwt_{sim_name}_{run+1}_obs"
        gwt_obs = [
            (f"conc_well_{i}", "CONCENTRATION", (lay, row, col))
            for i, (lay, row, col) in enumerate(obs_wells)
        ]

        flopy.mf6.ModflowUtlobs(
              gwt,
              pname=gwt_obs_package_name,
              digits=10,
              print_input=True,
              continuous=gwt_obs,

          )


   
        run_output_dir = os.path.join(output_dir, f"run_{run + 1}")
        os.makedirs(run_output_dir, exist_ok=True)

    
        concentrationfile = f"{sim_name}_{run}.ucn"  
        gwt_budgetfile = f"{sim_name}_{run}.cbc"
        flopy.mf6.ModflowGwtoc(gwt,
                               concentration_filerecord = [concentrationfile],
                               budget_filerecord = [gwt_budgetfile],
                               saverecord=[("CONCENTRATION", "ALL"), 
                                           ("BUDGET", "ALL")])

        flopy.mf6.ModflowGwfgwt(sim, exgtype='GWF6-GWT6', exgmnamea=gwfname, exgmnameb=gwtname,filename="{}.gwfgwt".format(sim_name))
       
        sim.write_simulation()
        success, buff = sim.run_simulation()
        if not success:
            raise Exception(f"模型运行失败 for run {run}")

已跳过重复的定水头单元格: (0, 2, 4)
已跳过重复的定水头单元格: (0, 4, 4)
已跳过重复的定水头单元格: (0, 6, 2)
已跳过重复的定水头单元格: (0, 28, 43)
已跳过重复的定水头单元格: (0, 26, 45)
已跳过重复的定水头单元格: (0, 24, 47)


KeyboardInterrupt: 

In [ ]:
import numpy as np
import pyDOE
import random
import os
import sys
import matplotlib.pyplot as plt
import flopy
import pandas as pd


def generate_initial_lhs(n, d, bounds):
    lhs = pyDOE.lhs(d, samples=n)
    X_init = np.zeros((n, d))
    for i in range(d):
        X_init[:, i] = bounds[i][0] + (bounds[i][1] - bounds[i][0]) * lhs[:, i]
    return X_init



def initialize_annealing_parameters():
    T0 = 1000
    alpha = 0.95
    Nmax = 1000
    return T0, alpha, Nmax



def cost_function(X):
    n = X.shape[0]
    min_dist = float('inf')
    for i in range(n):
        for j in range(i + 1, n):
            dist = np.linalg.norm(X[i] - X[j])
            min_dist = min(min_dist, dist)
    return min_dist



def generate_neighbor(X, mode='both'):
    n, d = X.shape
    X_candidate = X.copy()

    if mode == 'Q_only':
        # 仅扰动Q列（第0列）
        swap_indices = random.sample(range(n), max(1, n // 10))  # 扰动10%的样本
        np.random.shuffle(X_candidate[swap_indices, 0])

    elif mode == 'R_only':
        # 仅扰动R列（第1列）
        swap_indices = random.sample(range(n), max(1, n // 10))
        np.random.shuffle(X_candidate[swap_indices, 0])

    elif mode == 'both':
        # 同时扰动两个维度
        j = random.choice([0, 1])
        swap_indices = random.sample(range(n), max(1, n // 10))
        X_candidate[swap_indices, j] = np.random.permutation(X_candidate[swap_indices, j])

    return X_candidate



def metropolis_acceptance(DeltaC, T):
    if DeltaC > 0:
        return True
    else:
        return random.random() < np.exp(DeltaC / T)



def should_terminate(improvement_rates, T, T_min, Nmax, iteration):
    if len(improvement_rates) >= 10 and all(abs(rate) < 0.01 for rate in improvement_rates[-10:]):
        return True
    if T <= T_min:
        return True
    if iteration >= Nmax:
        return True
    return False


（增加模式参数）
def simulated_annealing(X_init, T0, alpha, Nmax, T_min=1, mode='both'):
    X_current = X_init
    C_current = cost_function(X_current)
    improvement_rates = []

    for iteration in range(Nmax):
        X_candidate = generate_neighbor(X_current, mode=mode)
        C_candidate = cost_function(X_candidate)
        DeltaC = C_candidate - C_current

        if metropolis_acceptance(DeltaC, T0):
            X_current = X_candidate
            C_current = C_candidate

        improvement_rate = (C_candidate - C_current) / C_current if C_current != 0 else 0
        improvement_rates.append(improvement_rate)

        if should_terminate(improvement_rates, T0, T_min, Nmax, iteration):
            break

        T0 = alpha * T0

    return X_current


output_dir = "confined_sample"
os.makedirs(output_dir, exist_ok=True)  # 自动创建目录（若不存在）

#定义参数
sim_name = "con"
length_units = "meters"
time_units = "days"


nper = 30  # 总模拟时间 10 年，分 20 个应力周期
nlay = 1
nrow = 30  # 假设值，需根据实际调整
ncol = 50  # 根据代码中最大列数调整
delr = 100.0
delc = 100.0
top = 100.0  # 假设值，需根据实际调整
botm = np.full((nlay, nrow, ncol), 60.0)  # 所有单元格层底高程为60m
icelltype = 1
strt = np.full((nlay, nrow, ncol), 70.0)  # 所有单元格初始水头70m

# 定水头边界条件
chd_spd = {
    0: [
        [0, row, 6, 100] for row in range(0, 2)
    ] + [
        [0, 2, col , 100] for col in range(4, 7)
    ] + [
        [0, row, 4, 100] for row in range(2, 5)
    ] + [
        [0, 6, col, 100] for col in range(2, 4)
    ] + [
        [0, row , 4, 100] for row in range(4, 7)
    ] + [
        [0, 8, col, 100] for col in range(0, 2)
    ] + [
        [0, row, 2, 100] for row in range(6, 9)
    ] + [
        [0, row, 41, 80] for row in range(28, 30)
    ] + [
        [0, 28, col, 80] for col in range(42, 44)
    ] + [
        [0, row , 43, 80] for row in range(26, 29)
    ] + [
        [0, 26, col, 80] for col in range(44, 46)
    ] + [
        [0, row, 45, 80] for row in range(24, 27)
    ] + [
        [0, 24, col, 80] for col in range(46, 48)
    ] + [
        [0, row, 47, 80] for row in range(22, 25)
    ] + [
        [0, 22, col, 80] for col in range(48, 50)
    ] 
}
unique_chd_spd = {0: []}
used_cells = set()
for item in chd_spd[0]:
    lay, row, col, _ = item
    cell_key = (lay, row, col)
    if cell_key not in used_cells:
        unique_chd_spd[0].append(item)
        used_cells.add(cell_key)
    else:
        print(f"已跳过重复的定水头单元格: ({lay}, {row}, {col})")

chd_spd = unique_chd_spd
# 水头边界和渗透系数分布相关代码
# 初始化 idomain 数组，所有单元格默认为活动状态
idomain = np.ones((nlay, nrow, ncol), dtype=np.int32)

# 原离散区域，修正了缺少逗号的问题
original_regions = [
    (0, 0, *range(0, 6)),
    (0, 2, *range(0, 4)),
    (0, 4, *range(0, 4)),
    (0, 6, *range(0, 2)),
    (0, 8, 0),
    (0, 12, *range(0, 2)),
    (0, 14, *range(0, 2)),
    (0, 16, *range(0, 4)),
    (0, 18, *range(0, 6)),
    (0, 20, *range(0, 8)),
    (0, 22, *range(0, 24)),
    (0, 24, *range(0, 26)),
    (0, 26, *range(0, 32)),
    (0, 28, *range(0, 34)),
    (0, 29, *range(0, 38))
]

# 存储连接后的区域
connected_regions = []

# 遍历原区域列表
for i in range(len(original_regions)):
    if i == 0:
        # 第一行直接添加
        connected_regions.append(original_regions[i])
    else:
        # 获取上一行和当前行的信息
        prev_row = original_regions[i - 1][1]
        current_row = original_regions[i][1]
        prev_cols = set(original_regions[i - 1][2:])
        current_cols = set(original_regions[i][2:])

        # 找出相邻两行列索引的最小和最大值
        min_col = min(min(prev_cols), min(current_cols))
        max_col = max(max(prev_cols), max(current_cols))

        # 填充上一行和当前行之间的空缺行
        for row in range(prev_row + 1, current_row):
            connected_regions.append((0, row, *range(min_col, max_col + 1)))

        # 添加当前行
        connected_regions.append(original_regions[i])


for region in connected_regions:
    lay = region[0]
    row = region[1]
    for col in region[2:]:
        idomain[lay, row, col] = -1


for row in range(8, 10):
    idomain[0, row, :2] = 1

original_regions = [
    (0, 0, *range(16, 50)),
    (0, 2, *range(20, 50)),
    (0, 4, *range(30, 50)),
    (0, 6, *range(32, 50)),
    (0, 8, *range(42, 50)),
    (0, 10, *range(46, 50)),
    (0, 12, *range(48, 50)),
    (0, 24, *range(48, 50)),
    (0, 26, *range(46, 50)),
    (0, 28, *range(44, 50)),
    (0, 29, *range(42, 50)),
]

# 存储连接后的区域
connected_regions = []

# 遍历原区域列表
for i in range(len(original_regions)):
    if i == 0:
        # 第一行直接添加
        connected_regions.append(original_regions[i])
    else:
        # 获取上一行和当前行的信息
        prev_row = original_regions[i - 1][1]
        current_row = original_regions[i][1]
        prev_cols = set(original_regions[i - 1][2:])
        current_cols = set(original_regions[i][2:])

        # 找出相邻两行列索引的最小和最大值
        min_col = min(min(prev_cols), min(current_cols))
        max_col = max(max(prev_cols), max(current_cols))

        # 填充上一行和当前行之间的空缺行
        for row in range(prev_row + 1, current_row):
            connected_regions.append((0, row, *range(min_col, max_col + 1)))

        # 添加当前行
        connected_regions.append(original_regions[i])


for region in connected_regions:
    lay = region[0]
    row = region[1]
    for col in region[2:]:
        idomain[lay, row, col] = -1


for row in range(14, 23):
    idomain[0, row, 48:] = 1
    
laytyp = 0  


hk_original = np.ones((nlay, nrow, ncol), dtype=np.float32)


hk_original[:, 0, 3:8] = 0.0004 * 864
hk_original[:, 1, 2:10] = 0.0004 * 864
hk_original[:, 2, 2:9] = 0.0004 * 864
hk_original[:, 3, 1:8] = 0.0004 * 864
hk_original[:, 4, 0:7] = 0.0004 * 864
hk_original[:, 5, 1:6] = 0.0004 * 864
hk_original[:, 6, 1:5] = 0.0004 * 864
hk_original[:, 7, 2:4] = 0.0004 * 864

hk_original[:, 2, 9:15] = 0.0002 * 864
hk_original[:, 3, 8:16] = 0.0002 * 864
hk_original[:, 4, 7:21] = 0.0002 * 864
hk_original[:, 5, 7:23] = 0.0002 * 864
hk_original[:, 6, 10:23] = 0.0002 * 864
hk_original[:, 7, 14:21] = 0.0002 * 864
hk_original[:, 8, 16:19] = 0.0002 * 864

hk_original[:, 5, 6] = 0.0001 * 864
hk_original[:, 6, 5:10] = 0.0001 * 864
hk_original[:, 7, 4:12] = 0.0001 * 864
hk_original[:, 8, 3:11] = 0.0001 * 864
hk_original[:, 9, 4:10] = 0.0001 * 864

hk_original[:, 7, 12:14] = 0.0003 * 864
hk_original[:, 8, 11:16] = 0.0003 * 864
hk_original[:, 9, 10:17] = 0.0003 * 864
hk_original[:, 10, 12:18] = 0.0003 * 864
hk_original[:, 11, 13:19] = 0.0003 * 864
hk_original[:, 12, 16:19] = 0.0003 * 864
hk_original[:, 13, 17:20] = 0.0003 * 864
hk_original[:, 14, 19] = 0.0003 * 864

hk_original[:, 6, 23] = 0.0007 * 864
hk_original[:, 7, 21:25] = 0.0007 * 864
hk_original[:, 8, 19:25] = 0.0007 * 864
hk_original[:, 9, 16:25] = 0.0007 * 864
hk_original[:, 10, 17:25] = 0.0007 * 864
hk_original[:, 11, 18:24] = 0.0007 * 864
hk_original[:, 12, 18:23] = 0.0007 * 864
hk_original[:, 13, 19:22] = 0.0007 * 864
hk_original[:, 14, 20] = 0.0007 * 864


hk = np.ones((nlay, nrow, ncol), dtype=np.float32)


row_scale = 2
col_scale = 2


for lay in range(nlay):
    for row in range(15):
        for col in range(25):
            new_row = row * row_scale
            new_col = col * col_scale
            hk[lay, new_row:new_row + row_scale, new_col:new_col + col_scale] = hk_original[lay, row, col]

num_stress_periods = 30  

rbot = np.linspace(90., 80.25, num=nrow)

riv_spd = {}
for sp in range(num_stress_periods):
    rstage = np.linspace(90.1, 81.25, num=nrow)
    riv_spd[sp] = []
    for col in [14]:
        for row in range(2, 11):      
            s = rstage[row - 2]
            b = rbot[row - 2]  
            riv_spd[sp].append([0, row, col, s, 50, b])


 6 个月
perlen = 6 * 30  
nstp = 1  
tsmult = 1  






permeability_values = [0.0004 * 864, 0.0002 * 864, 0.0001 * 864, 0.0003 * 864, 0.0007 * 864]


recharge = np.zeros((nlay, nrow, ncol))


evaporation = np.zeros((nlay, nrow, ncol))


rch_spd = {}
evt_spd = {}
for sp in range(nper):
    if sp % 2 == 1:  
       
        recharge_factors = {
            0.0004 * 864: np.random.uniform(0.016 * 1.1 / 100, 0.016 * 1.3 / 100),
            0.0002 * 864: np.random.uniform(0.012 * 1.1 / 100, 0.012 * 1.3 / 100),
            0.0001 * 864: np.random.uniform(0.008 * 1.1 / 100, 0.008 * 1.3 / 100),
            0.0003 * 864: np.random.uniform(0.014 * 1.1 / 100, 0.014 * 1.3 / 100),
            0.0007 * 864: np.random.uniform(0.018 * 1.1 / 100, 0.018 * 1.3 / 100)
        }
       
        evaporation_factors = {
            0.0004 * 864: np.random.uniform(0.002 * 0.7 / 100, 0.002 * 0.9 / 100),
            0.0002 * 864: np.random.uniform(0.003 * 0.7 / 100, 0.003 * 0.9 / 100),
            0.0001 * 864: np.random.uniform(0.001 * 0.7 / 100, 0.001 * 0.9 / 100),
            0.0003 * 864: np.random.uniform(0.004 * 0.7 / 100, 0.004 * 0.9 / 100),
            0.0007 * 864: np.random.uniform(0.005 * 0.7 / 100, 0.005 * 0.9 / 100)
        }
    else:  
       
        recharge_factors = {
            0.0004 * 864: np.random.uniform(0.016 * 0.7 / 100, 0.016 * 0.9 / 100),
            0.0002 * 864: np.random.uniform(0.012 * 0.7 / 100, 0.012 * 0.9 / 100),
            0.0001 * 864: np.random.uniform(0.008 * 0.7 / 100, 0.008 * 0.9 / 100),
            0.0003 * 864: np.random.uniform(0.014 * 0.7 / 100, 0.014 * 0.9 / 100),
            0.0007 * 864: np.random.uniform(0.018 * 0.7 / 100, 0.018 * 0.9 / 100)
        }

        evaporation_factors = {
            0.0004 * 864: np.random.uniform(0.002 * 1.1 / 100, 0.002 * 1.3 / 100),
            0.0002 * 864: np.random.uniform(0.003 * 1.1 / 100, 0.003 * 1.3 / 100),
            0.0001 * 864: np.random.uniform(0.001 * 1.1 / 100, 0.001 * 1.3 / 100),
            0.0003 * 864: np.random.uniform(0.004 * 1.1 / 100, 0.004 * 1.3 / 100),
            0.0007 * 864: np.random.uniform(0.005 * 1.1 / 100, 0.005 * 1.3 / 100)
        }

    current_rch = recharge.copy()
    current_evt = evaporation.copy()

    for k in permeability_values:
        rch_mask = np.isclose(hk, k)
        current_rch[rch_mask] = recharge_factors[k]

        evt_mask = np.isclose(hk, k)
        current_evt[evt_mask] = evaporation_factors[k]

    rch_spd[sp] = current_rch
    evt_spd[sp] = []
    for lay in range(nlay):
        for row in range(nrow):
            for col in range(ncol):
                if current_evt[lay, row, col] > 0:
                    lay = int(lay)
                    row = int(row)
                    col = int(col)
                    rate = float(current_evt[lay, row, col])
                    # 假设 surface 为 100.0（可根据实际情况修改）
                    surface = 100.0

                    depth = 5.0
                    evt_spd[sp].append([lay, row, col, surface, rate, depth])


# Solver parameters
nouter, ninner = 100, 300
hclose, rclose, relax = 1e-6, 1e-6, 1.0

ss = np.full((nlay, nrow, ncol), 1e-5)  
if __name__ == "__main__":
    n_Q = 4 
    d_Q = 10  
    bounds_Q = [(50, 100)] * d_Q  

    n_R_initial = 1  
    d_Ri = 20  
    bounds_Ri = [(90.10, 92.50)] * d_Ri  

    n_R_final = 1  
    d_Rf = 20  
    bounds_Rf = [(81.25, 82.50)] * d_Rf  
    for run in range(100):
 
        X_init_Q = generate_initial_lhs(n_Q, d_Q, bounds_Q)
        X_init_R_initial = generate_initial_lhs(n_R_initial, d_Ri, bounds_Ri)
        X_init_R_final = generate_initial_lhs(n_R_final, d_Rf, bounds_Rf)


        T0, alpha, Nmax = initialize_annealing_parameters()


        X_Q_optimized = simulated_annealing(X_init_Q, T0, alpha, Nmax, mode='Q_only')
        Q_optimized = X_Q_optimized


        X_R_initial_optimized = simulated_annealing(X_init_R_initial, T0, alpha, Nmax, mode='R_only')
        initial_water_level = X_R_initial_optimized


        X_R_final_optimized = simulated_annealing(X_init_R_final, T0, alpha, Nmax, mode='R_only')
        final_water_level = X_R_final_optimized


        for i in range(initial_water_level.shape[1]):
            if initial_water_level[0, i] < final_water_level[0, i]:
                initial_water_level[0, i], final_water_level[0, i] = final_water_level[0, i], initial_water_level[0, i]

        output_path = os.path.join(output_dir, f"optimized_Q_values_{run}.txt")
        np.savetxt(output_path, Q_optimized)
        output_path = os.path.join(output_dir, f"optimized_R_initial_values_{run}.txt")
        np.savetxt(output_path, initial_water_level)
        output_path = os.path.join(output_dir, f"optimized_R_final_values_{run}.txt")
        np.savetxt(output_path, final_water_level)


        n_wells = 10

        d_well = 3
        bounds_well = [(0, nlay - 1), (0, nrow - 1), (0, ncol - 1)]
 
        X_init_well = generate_initial_lhs(n_wells, d_well, bounds_well)

        X_init_well = np.round(X_init_well).astype(int)

        valid_well_locations = []
        for location in X_init_well:
            lay, row, col = location
            if idomain[lay, row, col] == 1:
                valid_well_locations.append((lay, row, col))
        while len(valid_well_locations) < n_wells:
            lay = np.random.randint(0, nlay)
            row = np.random.randint(0, nrow)
            col = np.random.randint(0, ncol)
            if idomain[lay, row, col] == 1:
                location = (lay, row, col)
                if location not in valid_well_locations:
                    valid_well_locations.append(location)

        X_init_well = np.array(valid_well_locations)


        T0, alpha, Nmax = initialize_annealing_parameters()


        X_well_optimized = simulated_annealing(X_init_well, T0, alpha, Nmax, mode='both')

        X_well_optimized = np.round(X_well_optimized).astype(int)
        well_locations = []
        for location in X_well_optimized:
            lay, row, col = location
            if idomain[lay, row, col] == 1:
                well_locations.append((lay, row, col))
        while len(well_locations) < n_wells:
            lay = np.random.randint(0, nlay)
            row = np.random.randint(0, nrow)
            col = np.random.randint(0, ncol)
            if idomain[lay, row, col] == 1:
                location = (lay, row, col)
                if location not in well_locations:
                    well_locations.append(location)

        c_values = np.array([0.5371, 0.9508, 0.8966, 0.9340, 
                0.6388 , 0.5806, 0.6135, 0.6934, 0.9771, 0.8470 ])
        c_values = c_values*20
        wel_spd = {
            i: [
                [lay, row, col, 0, 0] for lay, row, col in well_locations
            ] for i in range(4)
        }


        for stress_period in range(min(4, len(Q_optimized))):
            q_values = Q_optimized[stress_period]
            for i, well in enumerate(wel_spd[stress_period]):
                well[3] = q_values[i]
                well[4] = c_values[i]

        well_locations_array = np.array(well_locations)
        output_path_wells = os.path.join(output_dir, f"well_locations_{run}.txt")
        np.savetxt(output_path_wells, well_locations_array)
        

        sim_ws = os.path.join('confined_conc_models', f'{sim_name}_{run}') 
        sim = flopy.mf6.MFSimulation(sim_name=f'{sim_name}_{run}', sim_ws=sim_ws, exe_name='../modflow/mf6.6.1_linux/bin/mf6') 
         6 个月
        tdis_ds = [(perlen, nstp, tsmult) for _ in range(nper)] 
        flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_ds, time_units=time_units) 


        gwfname =  f"gwf_{sim_name}_{run}"
        gwf = flopy.mf6.ModflowGwf(sim, modelname=gwfname, newtonoptions="NEWTON UNDER_RELAXATION", 
                                   save_flows=True)  # newtonoptions="NEWTON UNDER_RELAXATION"​​牛顿-拉夫森法的松弛选项，用于改善非线性问题的收敛性

        imsgwf = flopy.mf6.ModflowIms(
            sim,
            print_option="SUMMARY",
            outer_dvclose=hclose,
            outer_maximum=nouter,
            under_relaxation="NONE",
            inner_maximum=ninner,
            inner_dvclose=hclose,
            rcloserecord=rclose,
            linear_acceleration="BICGSTAB",
            scaling_method="NONE",
            reordering_method="NONE",
            relaxation_factor=relax,
            filename="{}.ims".format(gwfname),
          )
        sim.register_ims_package(imsgwf, [gwf.name])

        flopy.mf6.ModflowGwfdis(gwf, length_units=length_units, nlay=nlay, nrow=nrow,
                                ncol=ncol, delr=delr, delc=delc, top=top, botm=botm, idomain=idomain)  

        flopy.mf6.ModflowGwfnpf(gwf, icelltype=icelltype, k=hk)  
        flopy.mf6.ModflowGwfsto(gwf, ss=ss, transient={0: True})
        flopy.mf6.ModflowGwfic(gwf, strt=strt)  。

        flopy.mf6.ModflowGwfriv(gwf, stress_period_data=riv_spd, pname="RIV-1", print_flows=True, save_flows=True)  # 定义河流边界条件。

        flopy.mf6.ModflowGwfwel(gwf, stress_period_data=wel_spd, auxiliary=['CONCENTRATION'], 
                                pname="WEL-1", save_flows=True)  

        flopy.mf6.ModflowGwfrcha(gwf, recharge=rch_spd)  
        flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_spd)  
        flopy.mf6.ModflowGwfevt(gwf, stress_period_data=evt_spd, pname="EVT-1", print_flows=True, save_flows=True)

        
        headfile = f"{sim_name}_{run}.hds"
        head_filerecord = [headfile]
        budgetfile = f"{sim_name}_{run}.bud"
        budget_filerecord = [budgetfile]
        saverecord = [("HEAD", "ALL"), ("BUDGET", "ALL")]
        flopy.mf6.ModflowGwfoc(gwf, saverecord=saverecord, head_filerecord=head_filerecord,
                               budget_filerecord=budget_filerecord, printrecord=saverecord)




        gwtname = f"gwt_{sim_name}_{run}"
        gwt = flopy.mf6.ModflowGwt(sim, modelname=gwtname, model_nam_file=f"{gwtname}.nam")


        imsgwt = flopy.mf6.ModflowIms(
            sim,
            print_option="SUMMARY",
            outer_dvclose=hclose,
            outer_maximum=nouter,
            under_relaxation="NONE",
            inner_maximum=ninner,
            inner_dvclose=hclose,
            rcloserecord=rclose,
            linear_acceleration="BICGSTAB",
            scaling_method="NONE",
            reordering_method="NONE",
            relaxation_factor=relax,
            filename="{}.ims".format(gwtname),
        )
        sim.register_ims_package(imsgwt, [gwt.name])


        flopy.mf6.ModflowGwtdis(gwt, nlay=nlay, nrow=nrow, ncol=ncol,
                                delr=delr, delc=delc, top=top, botm=botm, idomain=idomain)


        initial_concentration = 0.0  
        flopy.mf6.ModflowGwtic(gwt, strt=initial_concentration)


        dsp = flopy.mf6.ModflowGwtdsp(gwt,
                            alh=40.0,  
                            ath1=4.0,  
                            atv=0.1,   
                            diffc=0.0) 
   
        mst = flopy.mf6.ModflowGwtmst(gwt, porosity=0.3)  
        sourcerecarray = [('WEL-1', 'AUX', 'CONCENTRATION')]
        
        flopy.mf6.ModflowGwtssm(
              gwt,
              sources = sourcerecarray,
              print_flows=True,
              filename=f"{gwtname}.ssm"
         )


    
        gwt_obs_package_name = f"gwt_{sim_name}_{run+1}_obs"
        gwt_obs = [
            (f"conc_well_{i}", "CONCENTRATION", (lay, row, col))
            for i, (lay, row, col) in enumerate(obs_wells)
        ]

        flopy.mf6.ModflowUtlobs(
              gwt,
              pname=gwt_obs_package_name,
              digits=10,
              print_input=True,
              continuous=gwt_obs,

          )


   
        run_output_dir = os.path.join(output_dir, f"run_{run + 1}")
        os.makedirs(run_output_dir, exist_ok=True)

    
        concentrationfile = f"{sim_name}_{run}.ucn"  
        gwt_budgetfile = f"{sim_name}_{run}.cbc"
        flopy.mf6.ModflowGwtoc(gwt,
                               concentration_filerecord = [concentrationfile],
                               budget_filerecord = [gwt_budgetfile],
                               saverecord=[("CONCENTRATION", "ALL"), 
                                           ("BUDGET", "ALL")])
    
        flopy.mf6.ModflowGwfgwt(sim, exgtype='GWF6-GWT6', exgmnamea=gwfname, exgmnameb=gwtname,filename="{}.gwfgwt".format(sim_name))
       
        sim.write_simulation()
        success, buff = sim.run_simulation()
        if not success:
            raise Exception(f"模型运行失败 for run {run}")

In [ ]:
import numpy as np
import pyDOE
import random
import os
import sys
import matplotlib.pyplot as plt
import flopy
import pandas as pd


def generate_initial_lhs(n, d, bounds):
    lhs = pyDOE.lhs(d, samples=n)
    X_init = np.zeros((n, d))
    for i in range(d):
        X_init[:, i] = bounds[i][0] + (bounds[i][1] - bounds[i][0]) * lhs[:, i]
    return X_init


def initialize_annealing_parameters():
    T0 = 1000
    alpha = 0.95
    Nmax = 1000
    return T0, alpha, Nmax


def cost_function(X):
    n = X.shape[0]
    min_dist = float('inf')
    for i in range(n):
        for j in range(i + 1, n):
            dist = np.linalg.norm(X[i] - X[j])
            min_dist = min(min_dist, dist)
    return min_dist


def generate_neighbor(X, mode='both'):
    n, d = X.shape
    X_candidate = X.copy()

    if mode == 'Q_only':
        swap_indices = random.sample(range(n), max(1, n // 10))
        np.random.shuffle(X_candidate[swap_indices, 0])
    elif mode == 'R_only':
        swap_indices = random.sample(range(n), max(1, n // 10))
        np.random.shuffle(X_candidate[swap_indices, 0])
    elif mode == 'both':
        j = random.choice([0, 1])
        swap_indices = random.sample(range(n), max(1, n // 10))
        X_candidate[swap_indices, j] = np.random.permutation(X_candidate[swap_indices, j])
    return X_candidate


def metropolis_acceptance(DeltaC, T):
    if DeltaC > 0:
        return True
    else:
        return random.random() < np.exp(DeltaC / T)


def should_terminate(improvement_rates, T, T_min, Nmax, iteration):
    if len(improvement_rates) >= 10 and all(abs(rate) < 0.01 for rate in improvement_rates[-10:]):
        return True
    if T <= T_min:
        return True
    if iteration >= Nmax:
        return True
    return False


def simulated_annealing(X_init, T0, alpha, Nmax, T_min=1, mode='both'):
    X_current = X_init
    C_current = cost_function(X_current)
    improvement_rates = []

    for iteration in range(Nmax):
        X_candidate = generate_neighbor(X_current, mode=mode)
        C_candidate = cost_function(X_candidate)
        DeltaC = C_candidate - C_current

        if metropolis_acceptance(DeltaC, T0):
            X_current = X_candidate
            C_current = C_candidate

        improvement_rate = (C_candidate - C_current) / C_current if C_current != 0 else 0
        improvement_rates.append(improvement_rate)

        if should_terminate(improvement_rates, T0, T_min, Nmax, iteration):
            break

        T0 = alpha * T0
    return X_current


output_dir = "double_sample"
os.makedirs(output_dir, exist_ok=True)


sim_name = "dual"
length_units = "meters"
time_units = "days"


nper = 30
nlay = 2  
nrow = 30
ncol = 50
delr = 100.0
delc = 100.0
top = 100.0
botm = np.zeros((nlay, nrow, ncol))  
botm[0, :, :] = 80.0  
botm[1, :, :] = 60.0  
icelltype = [1, 0]  
laytyp = [1, 0]  
strt = np.full((nlay, nrow, ncol), 70.0)  
strt[0, :, :] = 90.0  
strt[1, :, :] = 70.0  


recharge = np.zeros((nrow, ncol))
evaporation = np.zeros(( nrow, ncol))


chd_spd = {
    0: [
        [0, row, 6, 100] for row in range(0, 2)
    ] + [
        [0, 2, col, 100] for col in range(4, 7)
    ] + [
        [0, row, 4, 100] for row in range(2, 5)
    ] + [
        [0, 6, col, 100] for col in range(2, 4)
    ] + [
        [0, row, 4, 100] for row in range(4, 7)
    ] + [
        [0, 8, col, 100] for col in range(0, 2)
    ] + [
        [0, row, 2, 100] for row in range(6, 9)
    ] + [
        [0, row, 41, 80] for row in range(28, 30)
    ] + [
        [0, 28, col, 80] for col in range(42, 44)
    ] + [
        [0, row, 43, 80] for row in range(26, 29)
    ] + [
        [0, 26, col, 80] for col in range(44, 46)
    ] + [
        [0, row, 45, 80] for row in range(24, 27)
    ] + [
        [0, 24, col, 80] for col in range(46, 48)
    ] + [
        [0, row, 47, 80] for row in range(22, 25)
    ] + [
        [0, 22, col, 80] for col in range(48, 50)
    ]
}
unique_chd_spd = {0: []}
used_cells = set()
for item in chd_spd[0]:
    lay, row, col, _ = item
    cell_key = (lay, row, col)
    if cell_key not in used_cells:
        unique_chd_spd[0].append(item)
        used_cells.add(cell_key)
    else:
        print(f"已跳过重复的定水头单元格: ({lay}, {row}, {col})")
chd_spd = unique_chd_spd


idomain = np.ones((nlay, nrow, ncol), dtype=np.int32)


original_regions = [
    (0, 0, *range(0, 6)),
    (0, 2, *range(0, 4)),
    (0, 4, *range(0, 4)),
    (0, 6, *range(0, 2)),
    (0, 8, 0),
    (0, 12, *range(0, 2)),
    (0, 14, *range(0, 2)),
    (0, 16, *range(0, 4)),
    (0, 18, *range(0, 6)),
    (0, 20, *range(0, 8)),
    (0, 22, *range(0, 24)),
    (0, 24, *range(0, 26)),
    (0, 26, *range(0, 32)),
    (0, 28, *range(0, 34)),
    (0, 29, *range(0, 38))
]

connected_regions = []
for i in range(len(original_regions)):
    if i == 0:
        connected_regions.append(original_regions[i])
    else:
        prev_row = original_regions[i - 1][1]
        current_row = original_regions[i][1]
        prev_cols = set(original_regions[i - 1][2:])
        current_cols = set(original_regions[i][2:])
        min_col = min(min(prev_cols), min(current_cols))
        max_col = max(max(prev_cols), max(current_cols))
        for row in range(prev_row + 1, current_row):
            connected_regions.append((0, row, *range(min_col, max_col + 1)))
        connected_regions.append(original_regions[i])

for region in connected_regions:
    lay = region[0]
    row = region[1]
    for col in region[2:]:
        idomain[lay, row, col] = -1
        idomain[1, row, col] = -1  

for row in range(8, 10):
    idomain[0, row, :2] = 1
    idomain[1, row, :2] = 1

original_regions = [
    (0, 0, *range(16, 50)),
    (0, 2, *range(20, 50)),
    (0, 4, *range(30, 50)),
    (0, 6, *range(32, 50)),
    (0, 8, *range(42, 50)),
    (0, 10, *range(46, 50)),
    (0, 12, *range(48, 50)),
    (0, 24, *range(48, 50)),
    (0, 26, *range(46, 50)),
    (0, 28, *range(44, 50)),
    (0, 29, *range(42, 50)),
]

connected_regions = []
for i in range(len(original_regions)):
    if i == 0:
        connected_regions.append(original_regions[i])
    else:
        prev_row = original_regions[i - 1][1]
        current_row = original_regions[i][1]
        prev_cols = set(original_regions[i - 1][2:])
        current_cols = set(original_regions[i][2:])
        min_col = min(min(prev_cols), min(current_cols))
        max_col = max(max(prev_cols), max(current_cols))
        for row in range(prev_row + 1, current_row):
            connected_regions.append((0, row, *range(min_col, max_col + 1)))
        connected_regions.append(original_regions[i])

for region in connected_regions:
    lay = region[0]
    row = region[1]
    for col in region[2:]:
        idomain[lay, row, col] = -1
        idomain[1, row, col] = -1

for row in range(14, 23):
    idomain[0, row, 48:] = 1
    idomain[1, row, 48:] = 1


hk = np.ones((nlay, nrow, ncol), dtype=np.float32)
hk_original = np.ones((1, 15, 25), dtype=np.float32)

hk_original[:, 0, 3:8] = 0.0004 * 864
hk_original[:, 1, 2:10] = 0.0004 * 864
hk_original[:, 2, 2:9] = 0.0004 * 864
hk_original[:, 3, 1:8] = 0.0004 * 864
hk_original[:, 4, 0:7] = 0.0004 * 864
hk_original[:, 5, 1:6] = 0.0004 * 864
hk_original[:, 6, 1:5] = 0.0004 * 864
hk_original[:, 7, 2:4] = 0.0004 * 864

hk_original[:, 2, 9:15] = 0.0002 * 864
hk_original[:, 3, 8:16] = 0.0002 * 864
hk_original[:, 4, 7:21] = 0.0002 * 864
hk_original[:, 5, 7:23] = 0.0002 * 864
hk_original[:, 6, 10:23] = 0.0002 * 864
hk_original[:, 7, 14:21] = 0.0002 * 864
hk_original[:, 8, 16:19] = 0.0002 * 864

hk_original[:, 5, 6] = 0.0001 * 864
hk_original[:, 6, 5:10] = 0.0001 * 864
hk_original[:, 7, 4:12] = 0.0001 * 864
hk_original[:, 8, 3:11] = 0.0001 * 864
hk_original[:, 9, 4:10] = 0.0001 * 864

hk_original[:, 7, 12:14] = 0.0003 * 864
hk_original[:, 8, 11:16] = 0.0003 * 864
hk_original[:, 9, 10:17] = 0.0003 * 864
hk_original[:, 10, 12:18] = 0.0003 * 864
hk_original[:, 11, 13:19] = 0.0003 * 864
hk_original[:, 12, 16:19] = 0.0003 * 864
hk_original[:, 13, 17:20] = 0.0003 * 864
hk_original[:, 14, 19] = 0.0003 * 864

hk_original[:, 6, 23] = 0.0007 * 864
hk_original[:, 7, 21:25] = 0.0007 * 864
hk_original[:, 8, 19:25] = 0.0007 * 864
hk_original[:, 9, 16:25] = 0.0007 * 864
hk_original[:, 10, 17:25] = 0.0007 * 864
hk_original[:, 11, 18:24] = 0.0007 * 864
hk_original[:, 12, 18:23] = 0.0007 * 864
hk_original[:, 13, 19:22] = 0.0007 * 864
hk_original[:, 14, 20] = 0.0007 * 864

row_scale = 2
col_scale = 2
for lay in range(nlay):
    for row in range(15):
        for col in range(25):
            new_row = row * row_scale
            new_col = col * col_scale
            hk[lay, new_row:new_row + row_scale, new_col:new_col + col_scale] = hk_original[0, row, col] * (0.8 if lay == 1 else 1.0)


sy = np.full((nlay, nrow, ncol), 0.2)  
sy[1, :, :] = 0.0  
ss = np.full((nlay, nrow, ncol), 1e-5)  


num_stress_periods = 30
rbot = np.linspace(90., 80.25, num=nrow)
riv_spd = {}
for sp in range(num_stress_periods):
    rstage = np.linspace(90.1, 81.25, num=nrow)
    riv_spd[sp] = []
    for col in [14]:
        for row in range(2, 11):
            s = rstage[row - 2]
            b = rbot[row - 2]
            riv_spd[sp].append([0, row, col, s, 50, b])  


perlen = 6 * 30
nstp = 1
tsmult = 1




permeability_values = [0.0004 * 864, 0.0002 * 864, 0.0001 * 864, 0.0003 * 864, 0.0007 * 864]


rch_spd = {}
evt_spd = {}
for sp in range(nper):
    if sp % 2 == 1:  
       
        recharge_factors = {
            0.0004 * 864: np.random.uniform(0.016 * 1.1 / 100, 0.016 * 1.3 / 100),
            0.0002 * 864: np.random.uniform(0.012 * 1.1 / 100, 0.012 * 1.3 / 100),
            0.0001 * 864: np.random.uniform(0.008 * 1.1 / 100, 0.008 * 1.3 / 100),
            0.0003 * 864: np.random.uniform(0.014 * 1.1 / 100, 0.014 * 1.3 / 100),
            0.0007 * 864: np.random.uniform(0.018 * 1.1 / 100, 0.018 * 1.3 / 100)
        }
       
        evaporation_factors = {
            0.0004 * 864: np.random.uniform(0.002 * 0.7 / 100, 0.002 * 0.9 / 100),
            0.0002 * 864: np.random.uniform(0.003 * 0.7 / 100, 0.003 * 0.9 / 100),
            0.0001 * 864: np.random.uniform(0.001 * 0.7 / 100, 0.001 * 0.9 / 100),
            0.0003 * 864: np.random.uniform(0.004 * 0.7 / 100, 0.004 * 0.9 / 100),
            0.0007 * 864: np.random.uniform(0.005 * 0.7 / 100, 0.005 * 0.9 / 100)
        }
    else:  
       
        recharge_factors = {
            0.0004 * 864: np.random.uniform(0.016 * 0.7 / 100, 0.016 * 0.9 / 100),
            0.0002 * 864: np.random.uniform(0.012 * 0.7 / 100, 0.012 * 0.9 / 100),
            0.0001 * 864: np.random.uniform(0.008 * 0.7 / 100, 0.008 * 0.9 / 100),
            0.0003 * 864: np.random.uniform(0.014 * 0.7 / 100, 0.014 * 0.9 / 100),
            0.0007 * 864: np.random.uniform(0.018 * 0.7 / 100, 0.018 * 0.9 / 100)
        }

        evaporation_factors = {
            0.0004 * 864: np.random.uniform(0.002 * 1.1 / 100, 0.002 * 1.3 / 100),
            0.0002 * 864: np.random.uniform(0.003 * 1.1 / 100, 0.003 * 1.3 / 100),
            0.0001 * 864: np.random.uniform(0.001 * 1.1 / 100, 0.001 * 1.3 / 100),
            0.0003 * 864: np.random.uniform(0.004 * 1.1 / 100, 0.004 * 1.3 / 100),
            0.0007 * 864: np.random.uniform(0.005 * 1.1 / 100, 0.005 * 1.3 / 100)
        }

    current_rch = recharge.copy()
    current_evt = evaporation.copy()


    assert hk[0].shape == (nrow, ncol), f"hk[0] shape mismatch: expected {(nrow, ncol)}, got {hk[0].shape}"
    assert current_rch.shape == (nrow, ncol), f"current_rch shape mismatch: expected {(nrow, ncol)}, got {current_rch.shape}"
    assert current_evt.shape == (nrow, ncol), f"current_evt shape mismatch: expected {(nrow, ncol)}, got {current_evt.shape}"

    for k in permeability_values:
        rch_mask = np.isclose(hk[0], k)
        evt_mask = np.isclose(hk[0], k)
        current_rch[rch_mask] = recharge_factors[k]
        current_evt[evt_mask] = evaporation_factors[k]

    rch_spd[sp] = current_rch  
    evt_spd[sp] = []
    for row in range(nrow):
        for col in range(ncol):
            if current_evt[row, col] > 0:
                rate = float(current_evt[row, col])
                surface = 100.0
                depth = 5.0
                evt_spd[sp].append([0, row, col, surface, rate, depth])


nouter, ninner = 100, 300
hclose, rclose, relax = 1e-6, 1e-6, 1.0

if __name__ == "__main__":
    n_Q = 4
    d_Q = 10
    bounds_Q = [(50, 100)] * d_Q
    n_R_initial = 1
    d_Ri = 20
    bounds_Ri = [(90.10, 92.50)] * d_Ri
    n_R_final = 1
    d_Rf = 20
    bounds_Rf = [(81.25, 82.50)] * d_Rf

    for run in range(100):
        X_init_Q = generate_initial_lhs(n_Q, d_Q, bounds_Q)
        X_init_R_initial = generate_initial_lhs(n_R_initial, d_Ri, bounds_Ri)
        X_init_R_final = generate_initial_lhs(n_R_final, d_Rf, bounds_Rf)

        T0, alpha, Nmax = initialize_annealing_parameters()

        X_Q_optimized = simulated_annealing(X_init_Q, T0, alpha, Nmax, mode='Q_only')
        Q_optimized = X_Q_optimized

        X_R_initial_optimized = simulated_annealing(X_init_R_initial, T0, alpha, Nmax, mode='R_only')
        initial_water_level = X_R_initial_optimized

        X_R_final_optimized = simulated_annealing(X_init_R_final, T0, alpha, Nmax, mode='R_only')
        final_water_level = X_R_final_optimized

        for i in range(initial_water_level.shape[1]):
            if initial_water_level[0, i] < final_water_level[0, i]:
                initial_water_level[0, i], final_water_level[0, i] = final_water_level[0, i], initial_water_level[0, i]

        output_path = os.path.join(output_dir, f"optimized_Q_values_{run}.txt")
        np.savetxt(output_path, Q_optimized)
        output_path = os.path.join(output_dir, f"optimized_R_initial_values_{run}.txt")
        np.savetxt(output_path, initial_water_level)
        output_path = os.path.join(output_dir, f"optimized_R_final_values_{run}.txt")
        np.savetxt(output_path, final_water_level)

        n_wells = 10
        d_well = 3
        bounds_well = [(0, nlay - 1), (0, nrow - 1), (0, ncol - 1)]
        X_init_well = generate_initial_lhs(n_wells, d_well, bounds_well)
        X_init_well = np.round(X_init_well).astype(int)
        valid_well_locations = []
        for location in X_init_well:
            lay, row, col = location
            if idomain[lay, row, col] == 1:
                valid_well_locations.append((lay, row, col))
        while len(valid_well_locations) < n_wells:
            lay = np.random.randint(0, nlay)
            row = np.random.randint(0, nrow)
            col = np.random.randint(0, ncol)
            if idomain[lay, row, col] == 1:
                location = (lay, row, col)
                if location not in valid_well_locations:
                    valid_well_locations.append(location)
        X_init_well = np.array(valid_well_locations)

        T0, alpha, Nmax = initialize_annealing_parameters()
        X_well_optimized = simulated_annealing(X_init_well, T0, alpha, Nmax, mode='both')
        X_well_optimized = np.round(X_well_optimized).astype(int)
        well_locations = []
        for location in X_well_optimized:
            lay, row, col = location
            if idomain[lay, row, col] == 1:
                well_locations.append((lay, row, col))
        while len(well_locations) < n_wells:
            lay = np.random.randint(0, nlay)
            row = np.random.randint(0, nrow)
            col = np.random.randint(0, ncol)
            if idomain[lay, row, col] == 1:
                location = (lay, row, col)
                if location not in well_locations:
                    well_locations.append(location)

        c_values = np.array([0.5371, 0.9508, 0.8966, 0.9340, 0.6388, 0.5806, 0.6135, 0.6934, 0.9771, 0.8470])
        c_values = c_values*20
        wel_spd = {
            i: [
                [lay, row, col, 0, 0] for lay, row, col in well_locations
            ] for i in range(4)
        }

        for stress_period in range(min(4, len(Q_optimized))):
            q_values = Q_optimized[stress_period]
            for i, well in enumerate(wel_spd[stress_period]):
                well[3] = q_values[i]
                well[4] = c_values[i]

        well_locations_array = np.array(well_locations)
        output_path_wells = os.path.join(output_dir, f"well_locations_{run}.txt")
        np.savetxt(output_path_wells, well_locations_array)

        sim_ws = os.path.join('double_conc_models', f'{sim_name}_{run}')
        sim = flopy.mf6.MFSimulation(sim_name=f'{sim_name}_{run}', sim_ws=sim_ws, exe_name='../modflow/mf6.6.1_linux/bin/mf6')

        tdis_ds = [(perlen, nstp, tsmult) for _ in range(nper)]
        flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_ds, time_units=time_units)

        gwfname = f"gwf_{sim_name}_{run}"
        gwf = flopy.mf6.ModflowGwf(sim, modelname=gwfname, newtonoptions="NEWTON UNDER_RELAXATION", save_flows=True)
        imsgwf = flopy.mf6.ModflowIms(
            sim,
            print_option="SUMMARY",
            outer_dvclose=hclose,
            outer_maximum=nouter,
            under_relaxation="NONE",
            inner_maximum=ninner,
            inner_dvclose=hclose,
            rcloserecord=rclose,
            linear_acceleration="BICGSTAB",
            scaling_method="NONE",
            reordering_method="NONE",
            relaxation_factor=relax,
            filename="{}.ims".format(gwfname),
        )
        sim.register_ims_package(imsgwf, [gwf.name])

        flopy.mf6.ModflowGwfdis(gwf, length_units=length_units, nlay=nlay, nrow=nrow,
                                ncol=ncol, delr=delr, delc=delc, top=top, botm=botm, idomain=idomain)
        flopy.mf6.ModflowGwfnpf(gwf, icelltype=icelltype, k=hk, save_specific_discharge=True)
        flopy.mf6.ModflowGwfsto(gwf, sy=sy, ss=ss, transient={0: True})
        flopy.mf6.ModflowGwfic(gwf, strt=strt)
        flopy.mf6.ModflowGwfriv(gwf, stress_period_data=riv_spd, pname="RIV-1", print_flows=True, save_flows=True)
        flopy.mf6.ModflowGwfwel(gwf, stress_period_data=wel_spd, auxiliary=['CONCENTRATION'], pname="WEL-1", save_flows=True)
        flopy.mf6.ModflowGwfrcha(gwf, recharge=rch_spd)
        flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_spd)
        flopy.mf6.ModflowGwfevt(gwf, stress_period_data=evt_spd, pname="EVT-1", print_flows=True, save_flows=True)

        headfile = f"{sim_name}_{run}.hds"
        budgetfile = f"{sim_name}_{run}.bud"
        saverecord = [("HEAD", "ALL"), ("BUDGET", "ALL")]
        flopy.mf6.ModflowGwfoc(gwf, saverecord=saverecord, head_filerecord=[headfile],
                               budget_filerecord=[budgetfile], printrecord=saverecord)

        obs_data = {
            f'obs_well_{i+1}.csv': [
                (f'well_{i+1}', 'HEAD', (lay, row, col))
            ] for i, (lay, row, col) in enumerate(obs_wells)
        }

        flopy.mf6.ModflowUtlobs(
            gwf,
            digits=10,
            print_input=True,
            continuous=obs_data,
        )

        gwtname = f"gwt_{sim_name}_{run}"
        gwt = flopy.mf6.ModflowGwt(sim, modelname=gwtname, model_nam_file=f"{gwtname}.nam")
        imsgwt = flopy.mf6.ModflowIms(
            sim,
            print_option="SUMMARY",
            outer_dvclose=hclose,
            outer_maximum=nouter,
            under_relaxation="NONE",
            inner_maximum=ninner,
            inner_dvclose=hclose,
            rcloserecord=rclose,
            linear_acceleration="BICGSTAB",
            scaling_method="NONE",
            reordering_method="NONE",
            relaxation_factor=relax,
            filename="{}.ims".format(gwtname),
        )
        sim.register_ims_package(imsgwt, [gwt.name])

        flopy.mf6.ModflowGwtdis(gwt, nlay=nlay, nrow=nrow, ncol=ncol,
                                delr=delr, delc=delc, top=top, botm=botm, idomain=idomain)
        initial_concentration = 0.0
        flopy.mf6.ModflowGwtic(gwt, strt=initial_concentration)
        flopy.mf6.ModflowGwtdsp(gwt, alh=40.0, ath1=4.0, atv=0.1, diffc=0.0)
        flopy.mf6.ModflowGwtmst(gwt, porosity=0.3)
        sourcerecarray = [('WEL-1', 'AUX', 'CONCENTRATION')]
        flopy.mf6.ModflowGwtssm(gwt, sources=sourcerecarray, print_flows=True, filename=f"{gwtname}.ssm")

        gwt_obs = [
            (f"conc_well_{i}", "CONCENTRATION", (lay, row, col))
            for i, (lay, row, col) in enumerate(obs_wells)
        ]
        flopy.mf6.ModflowUtlobs(gwt, pname=f"gwt_{sim_name}_{run}_obs", digits=10, print_input=True, continuous=gwt_obs)

        run_output_dir = os.path.join(output_dir, f"run_{run + 1}")
        os.makedirs(run_output_dir, exist_ok=True)

        concentrationfile = f"{sim_name}_{run}.ucn"
        gwt_budgetfile = f"{sim_name}_{run}.cbc"
        flopy.mf6.ModflowGwtoc(gwt, concentration_filerecord=[concentrationfile],
                               budget_filerecord=[gwt_budgetfile],
                               saverecord=[("CONCENTRATION", "ALL"), ("BUDGET", "ALL")])
        flopy.mf6.ModflowGwfgwt(sim, exgtype='GWF6-GWT6', exgmnamea=gwfname, exgmnameb=gwtname, filename="{}.gwfgwt".format(sim_name))

        sim.write_simulation()
        success, buff = sim.run_simulation()
        if not success:
            raise Exception(f"模型运行失败 for run {run}")

In [ ]:
import numpy as np
import pyDOE
import random
import os
import sys
import matplotlib.pyplot as plt
import flopy
import pandas as pd
import gstools as gs 


def generate_initial_lhs(n, d, bounds):
    lhs = pyDOE.lhs(d, samples=n)
    X_init = np.zeros((n, d))
    for i in range(d):
        X_init[:, i] = bounds[i][0] + (bounds[i][1] - bounds[i][0]) * lhs[:, i]
    return X_init


def initialize_annealing_parameters():
    T0 = 1000
    alpha = 0.95
    Nmax = 1000
    return T0, alpha, Nmax


def cost_function(X):
    n = X.shape[0]
    min_dist = float('inf')
    for i in range(n):
        for j in range(i + 1, n):
            dist = np.linalg.norm(X[i] - X[j])
            min_dist = min(min_dist, dist)
    return min_dist


def generate_neighbor(X, mode='both'):
    n, d = X.shape
    X_candidate = X.copy()

    if mode == 'Q_only':
        swap_indices = random.sample(range(n), max(1, n // 10))
        np.random.shuffle(X_candidate[swap_indices, 0])
    elif mode == 'R_only':
        swap_indices = random.sample(range(n), max(1, n // 10))
        np.random.shuffle(X_candidate[swap_indices, 0])
    elif mode == 'both':
        j = random.choice([0, 1])
        swap_indices = random.sample(range(n), max(1, n // 10))
        X_candidate[swap_indices, j] = np.random.permutation(X_candidate[swap_indices, j])
    return X_candidate


def metropolis_acceptance(DeltaC, T):
    if DeltaC > 0:
        return True
    else:
        return random.random() < np.exp(DeltaC / T)


def should_terminate(improvement_rates, T, T_min, Nmax, iteration):
    if len(improvement_rates) >= 10 and all(abs(rate) < 0.01 for rate in improvement_rates[-10:]):
        return True
    if T <= T_min:
        return True
    if iteration >= Nmax:
        return True
    return False


def simulated_annealing(X_init, T0, alpha, Nmax, T_min=1, mode='both'):
    X_current = X_init
    C_current = cost_function(X_current)
    improvement_rates = []

    for iteration in range(Nmax):
        X_candidate = generate_neighbor(X_current, mode=mode)
        C_candidate = cost_function(X_candidate)
        DeltaC = C_candidate - C_current

        if metropolis_acceptance(DeltaC, T0):
            X_current = X_candidate
            C_current = C_candidate

        improvement_rate = (C_candidate - C_current) / C_current if C_current != 0 else 0
        improvement_rates.append(improvement_rate)

        if should_terminate(improvement_rates, T0, T_min, Nmax, iteration):
            break

        T0 = alpha * T0
    return X_current


output_dir = "dual_guass_sample"
os.makedirs(output_dir, exist_ok=True)


sim_name = "dual"
length_units = "meters"
time_units = "days"


nper = 30
nlay = 2
nrow = 30
ncol = 50
delr = 100.0
delc = 100.0


def generate_grf(nrow, ncol, mean, variance, correlation_length, model='exponential'):
    x = np.arange(0, ncol * delc, delc)
    y = np.arange(0, nrow * delr, delr)
    model = gs.Exponential(dim=2, var=variance, len_scale=correlation_length)
    srf = gs.SRF(model, mean=mean)
    field = srf.structured([y, x])
    return field

# 生成 top 场
top_mean = 100.0
top_variance = 5.0  
correlation_length = 500.0  
top = generate_grf(nrow, ncol, top_mean, top_variance, correlation_length)
top = np.clip(top, 95.0, 105.0) 

# 生成 botm 场
botm = np.zeros((nlay, nrow, ncol))
botm_mean = [80.0, 60.0]  
botm_variance = [3.0, 3.0]  
for lay in range(nlay):
    botm[lay] = generate_grf(nrow, ncol, botm_mean[lay], botm_variance[lay], correlation_length)
    if lay == 0:
        botm[lay] = np.clip(botm[lay], 75.0, np.minimum(top - 5.0, 80.0))
    else:
        botm[lay] = np.clip(botm[lay], 55.0, botm[lay-1] - 5.0)  # 确保 botm[1] < botm[0]


hk_mean = np.log(0.0004 * 864)  
hk_variance = 1.0  
hk = np.zeros((nlay, nrow, ncol))
for lay in range(nlay):
    log_hk = generate_grf(nrow, ncol, hk_mean, hk_variance, correlation_length)
    hk[lay] = np.exp(log_hk) * (0.8 if lay == 1 else 1.0)  
    hk[lay] = np.clip(hk[lay], 0.0001 * 864, 0.001 * 864)  


idomain = np.ones((nlay, nrow, ncol), dtype=np.int32)
original_regions = [
    (0, 0, *range(0, 6)),
    (0, 2, *range(0, 4)),
    (0, 4, *range(0, 4)),
    (0, 6, *range(0, 2)),
    (0, 8, 0),
    (0, 12, *range(0, 2)),
    (0, 14, *range(0, 2)),
    (0, 16, *range(0, 4)),
    (0, 18, *range(0, 6)),
    (0, 20, *range(0, 8)),
    (0, 22, *range(0, 24)),
    (0, 24, *range(0, 26)),
    (0, 26, *range(0, 32)),
    (0, 28, *range(0, 34)),
    (0, 29, *range(0, 38))
]

connected_regions = []
for i in range(len(original_regions)):
    if i == 0:
        connected_regions.append(original_regions[i])
    else:
        prev_row = original_regions[i - 1][1]
        current_row = original_regions[i][1]
        prev_cols = set(original_regions[i - 1][2:])
        current_cols = set(original_regions[i][2:])
        min_col = min(min(prev_cols), min(current_cols))
        max_col = max(max(prev_cols), max(current_cols))
        for row in range(prev_row + 1, current_row):
            connected_regions.append((0, row, *range(min_col, max_col + 1)))
        connected_regions.append(original_regions[i])

for region in connected_regions:
    lay = region[0]
    row = region[1]
    for col in region[2:]:
        idomain[lay, row, col] = -1
        idomain[1, row, col] = -1

for row in range(8, 10):
    idomain[0, row, :2] = 1
    idomain[1, row, :2] = 1

original_regions = [
    (0, 0, *range(16, 50)),
    (0, 2, *range(20, 50)),
    (0, 4, *range(30, 50)),
    (0, 6, *range(32, 50)),
    (0, 8, *range(42, 50)),
    (0, 10, *range(46, 50)),
    (0, 12, *range(48, 50)),
    (0, 24, *range(48, 50)),
    (0, 26, *range(46, 50)),
    (0, 28, *range(44, 50)),
    (0, 29, *range(42, 50)),
]

connected_regions = []
for i in range(len(original_regions)):
    if i == 0:
        connected_regions.append(original_regions[i])
    else:
        prev_row = original_regions[i - 1][1]
        current_row = original_regions[i][1]
        prev_cols = set(original_regions[i - 1][2:])
        current_cols = set(original_regions[i][2:])
        min_col = min(min(prev_cols), min(current_cols))
        max_col = max(max(prev_cols), max(current_cols))
        for row in range(prev_row + 1, current_row):
            connected_regions.append((0, row, *range(min_col, max_col + 1)))
        connected_regions.append(original_regions[i])

for region in connected_regions:
    lay = region[0]
    row = region[1]
    for col in region[2:]:
        idomain[lay, row, col] = -1
        idomain[1, row, col] = -1

for row in range(14, 23):
    idomain[0, row, 48:] = 1
    idomain[1, row, 48:] = 1


top[idomain[0] == -1] = np.nan
for lay in range(nlay):
    botm[lay][idomain[lay] == -1] = np.nan
    hk[lay][idomain[lay] == -1] = np.nan


icelltype = [1, 0]
laytyp = [1, 0]
strt = np.full((nlay, nrow, ncol), 70.0)
strt[0, :, :] = 90.0
strt[1, :, :] = 70.0

recharge = np.zeros((nrow, ncol))
evaporation = np.zeros((nrow, ncol))

chd_spd = {
    0: [
        [0, row, 6, 100] for row in range(0, 2)
    ] + [
        [0, 2, col, 100] for col in range(4, 7)
    ] + [
        [0, row, 4, 100] for row in range(2, 5)
    ] + [
        [0, 6, col, 100] for col in range(2, 4)
    ] + [
        [0, row, 4, 100] for row in range(4, 7)
    ] + [
        [0, 8, col, 100] for col in range(0, 2)
    ] + [
        [0, row, 2, 100] for row in range(6, 9)
    ] + [
        [0, row, 41, 80] for row in range(28, 30)
    ] + [
        [0, 28, col, 80] for col in range(42, 44)
    ] + [
        [0, row, 43, 80] for row in range(26, 29)
    ] + [
        [0, 26, col, 80] for col in range(44, 46)
    ] + [
        [0, row, 45, 80] for row in range(24, 27)
    ] + [
        [0, 24, col, 80] for col in range(46, 48)
    ] + [
        [0, row, 47, 80] for row in range(22, 25)
    ] + [
        [0, 22, col, 80] for col in range(48, 50)
    ]
}
unique_chd_spd = {0: []}
used_cells = set()
for item in chd_spd[0]:
    lay, row, col, _ = item
    cell_key = (lay, row, col)
    if cell_key not in used_cells:
        unique_chd_spd[0].append(item)
        used_cells.add(cell_key)
    else:
        print(f"已跳过重复的定水头单元格: ({lay}, {row}, {col})")
        
chd_spd = unique_chd_spd
sy = np.full((nlay, nrow, ncol), 0.2)  
sy[1, :, :] = 0.0  
ss = np.full((nlay, nrow, ncol), 1e-5)  

num_stress_periods = 30
rbot = np.linspace(90., 80.25, num=nrow)
riv_spd = {}
for sp in range(num_stress_periods):
    rstage = np.linspace(90.1, 81.25, num=nrow)
    riv_spd[sp] = []
    for col in [14]:
        for row in range(2, 11):
            s = rstage[row - 2]
            b = rbot[row - 2]
            riv_spd[sp].append([0, row, col, s, 50, b])


perlen = 6 * 30
nstp = 1
tsmult = 1




permeability_values = [0.0001 * 864, 0.0002 * 864, 0.0003 * 864, 0.0004 * 864, 0.0007 * 864]


rch_spd = {}
evt_spd = {}
for sp in range(nper):
    if sp % 2 == 1:  
       
        recharge_factors = {
            0.0004 * 864: np.random.uniform(0.016 * 1.1 / 100, 0.016 * 1.3 / 100),
            0.0002 * 864: np.random.uniform(0.012 * 1.1 / 100, 0.012 * 1.3 / 100),
            0.0001 * 864: np.random.uniform(0.008 * 1.1 / 100, 0.008 * 1.3 / 100),
            0.0003 * 864: np.random.uniform(0.014 * 1.1 / 100, 0.014 * 1.3 / 100),
            0.0007 * 864: np.random.uniform(0.018 * 1.1 / 100, 0.018 * 1.3 / 100)
        }
       
        evaporation_factors = {
            0.0004 * 864: np.random.uniform(0.002 * 0.7 / 100, 0.002 * 0.9 / 100),
            0.0002 * 864: np.random.uniform(0.003 * 0.7 / 100, 0.003 * 0.9 / 100),
            0.0001 * 864: np.random.uniform(0.001 * 0.7 / 100, 0.001 * 0.9 / 100),
            0.0003 * 864: np.random.uniform(0.004 * 0.7 / 100, 0.004 * 0.9 / 100),
            0.0007 * 864: np.random.uniform(0.005 * 0.7 / 100, 0.005 * 0.9 / 100)
        }
    else:  
       
        recharge_factors = {
            0.0004 * 864: np.random.uniform(0.016 * 0.7 / 100, 0.016 * 0.9 / 100),
            0.0002 * 864: np.random.uniform(0.012 * 0.7 / 100, 0.012 * 0.9 / 100),
            0.0001 * 864: np.random.uniform(0.008 * 0.7 / 100, 0.008 * 0.9 / 100),
            0.0003 * 864: np.random.uniform(0.014 * 0.7 / 100, 0.014 * 0.9 / 100),
            0.0007 * 864: np.random.uniform(0.018 * 0.7 / 100, 0.018 * 0.9 / 100)
        }

        evaporation_factors = {
            0.0004 * 864: np.random.uniform(0.002 * 1.1 / 100, 0.002 * 1.3 / 100),
            0.0002 * 864: np.random.uniform(0.003 * 1.1 / 100, 0.003 * 1.3 / 100),
            0.0001 * 864: np.random.uniform(0.001 * 1.1 / 100, 0.001 * 1.3 / 100),
            0.0003 * 864: np.random.uniform(0.004 * 1.1 / 100, 0.004 * 1.3 / 100),
            0.0007 * 864: np.random.uniform(0.005 * 1.1 / 100, 0.005 * 1.3 / 100)
        }

    current_rch = recharge.copy()
    current_evt = evaporation.copy()

    for k in permeability_values:
        rch_mask = np.abs(hk[0] - k) < 1e-6 * 864 
        evt_mask = np.abs(hk[0] - k) < 1e-6 * 864
        current_rch[rch_mask] = recharge_factors[k]
        current_evt[evt_mask] = evaporation_factors[k]

    rch_spd[sp] = current_rch
    evt_spd[sp] = []
    for row in range(nrow):
        for col in range(ncol):
            if current_evt[row, col] > 0:
                rate = float(current_evt[row, col])
                surface = top[row, col] if not np.isnan(top[row, col]) else 100.0
                depth = 5.0
                evt_spd[sp].append([0, row, col, surface, rate, depth])


nouter, ninner = 100, 300
hclose, rclose, relax = 1e-6, 1e-6, 1.0

if __name__ == "__main__":
    n_Q = 4
    d_Q = 10
    bounds_Q = [(50, 100)] * d_Q
    n_R_initial = 1
    d_Ri = 20
    bounds_Ri = [(90.10, 92.50)] * d_Ri
    n_R_final = 1
    d_Rf = 20
    bounds_Rf = [(81.25, 82.50)] * d_Rf

    for run in range(100):
        X_init_Q = generate_initial_lhs(n_Q, d_Q, bounds_Q)
        X_init_R_initial = generate_initial_lhs(n_R_initial, d_Ri, bounds_Ri)
        X_init_R_final = generate_initial_lhs(n_R_final, d_Rf, bounds_Rf)

        T0, alpha, Nmax = initialize_annealing_parameters()

        X_Q_optimized = simulated_annealing(X_init_Q, T0, alpha, Nmax, mode='Q_only')
        Q_optimized = X_Q_optimized

        X_R_initial_optimized = simulated_annealing(X_init_R_initial, T0, alpha, Nmax, mode='R_only')
        initial_water_level = X_R_initial_optimized

        X_R_final_optimized = simulated_annealing(X_init_R_final, T0, alpha, Nmax, mode='R_only')
        final_water_level = X_R_final_optimized

        for i in range(initial_water_level.shape[1]):
            if initial_water_level[0, i] < final_water_level[0, i]:
                initial_water_level[0, i], final_water_level[0, i] = final_water_level[0, i], initial_water_level[0, i]

        output_path = os.path.join(output_dir, f"optimized_Q_values_{run}.txt")
        np.savetxt(output_path, Q_optimized)
        output_path = os.path.join(output_dir, f"optimized_R_initial_values_{run}.txt")
        np.savetxt(output_path, initial_water_level)
        output_path = os.path.join(output_dir, f"optimized_R_final_values_{run}.txt")
        np.savetxt(output_path, final_water_level)

        n_wells = 10
        d_well = 3
        bounds_well = [(0, nlay - 1), (0, nrow - 1), (0, ncol - 1)]
        X_init_well = generate_initial_lhs(n_wells, d_well, bounds_well)
        X_init_well = np.round(X_init_well).astype(int)
        valid_well_locations = []
        for location in X_init_well:
            lay, row, col = location
            if idomain[lay, row, col] == 1:
                valid_well_locations.append((lay, row, col))
        while len(valid_well_locations) < n_wells:
            lay = np.random.randint(0, nlay)
            row = np.random.randint(0, nrow)
            col = np.random.randint(0, ncol)
            if idomain[lay, row, col] == 1:
                location = (lay, row, col)
                if location not in valid_well_locations:
                    valid_well_locations.append(location)
        X_init_well = np.array(valid_well_locations)

        T0, alpha, Nmax = initialize_annealing_parameters()
        X_well_optimized = simulated_annealing(X_init_well, T0, alpha, Nmax, mode='both')
        X_well_optimized = np.round(X_well_optimized).astype(int)
        well_locations = []
        for location in X_well_optimized:
            lay, row, col = location
            if idomain[lay, row, col] == 1:
                well_locations.append((lay, row, col))
        while len(well_locations) < n_wells:
            lay = np.random.randint(0, nlay)
            row = np.random.randint(0, nrow)
            col = np.random.randint(0, ncol)
            if idomain[lay, row, col] == 1:
                location = (lay, row, col)
                if location not in well_locations:
                    well_locations.append(location)

        c_values = np.array([0.5371, 0.9508, 0.8966, 0.9340, 0.6388, 0.5806, 0.6135, 0.6934, 0.9771, 0.8470])
        c_values = c_values*20
        wel_spd = {
            i: [
                [lay, row, col, 0, 0] for lay, row, col in well_locations
            ] for i in range(4)
        }

        for stress_period in range(min(4, len(Q_optimized))):
            q_values = Q_optimized[stress_period]
            for i, well in enumerate(wel_spd[stress_period]):
                well[3] = q_values[i]
                well[4] = c_values[i]

        well_locations_array = np.array(well_locations)
        output_path_wells = os.path.join(output_dir, f"well_locations_{run}.txt")
        np.savetxt(output_path_wells, well_locations_array)

        sim_ws = os.path.join('dual_guass_conc_models', f'{sim_name}_{run}')
        sim = flopy.mf6.MFSimulation(sim_name=f'{sim_name}_{run}', sim_ws=sim_ws, exe_name='../modflow/mf6.6.1_linux/bin/mf6')

        tdis_ds = [(perlen, nstp, tsmult) for _ in range(nper)]
        flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_ds, time_units=time_units)

        gwfname = f"gwf_{sim_name}_{run}"
        gwf = flopy.mf6.ModflowGwf(sim, modelname=gwfname, newtonoptions="NEWTON UNDER_RELAXATION", save_flows=True)
        imsgwf = flopy.mf6.ModflowIms(
            sim,
            print_option="SUMMARY",
            outer_dvclose=hclose,
            outer_maximum=nouter,
            under_relaxation="NONE",
            inner_maximum=ninner,
            inner_dvclose=hclose,
            rcloserecord=rclose,
            linear_acceleration="BICGSTAB",
            scaling_method="NONE",
            reordering_method="NONE",
            relaxation_factor=relax,
            filename="{}.ims".format(gwfname),
        )
        sim.register_ims_package(imsgwf, [gwf.name])

        flopy.mf6.ModflowGwfdis(gwf, length_units=length_units, nlay=nlay, nrow=nrow,
                                ncol=ncol, delr=delr, delc=delc, top=top, botm=botm, idomain=idomain)
        flopy.mf6.ModflowGwfnpf(gwf, icelltype=icelltype, k=hk, save_specific_discharge=True)
        flopy.mf6.ModflowGwfsto(gwf, sy=sy, ss=ss, transient={0: True})
        flopy.mf6.ModflowGwfic(gwf, strt=strt)
        flopy.mf6.ModflowGwfriv(gwf, stress_period_data=riv_spd, pname="RIV-1", print_flows=True, save_flows=True)
        flopy.mf6.ModflowGwfwel(gwf, stress_period_data=wel_spd, auxiliary=['CONCENTRATION'], pname="WEL-1", save_flows=True)
        flopy.mf6.ModflowGwfrcha(gwf, recharge=rch_spd)
        flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_spd)
        flopy.mf6.ModflowGwfevt(gwf, stress_period_data=evt_spd, pname="EVT-1", print_flows=True, save_flows=True)

        headfile = f"{sim_name}_{run}.hds"
        budgetfile = f"{sim_name}_{run}.bud"
        saverecord = [("HEAD", "ALL"), ("BUDGET", "ALL")]
        flopy.mf6.ModflowGwfoc(gwf, saverecord=saverecord, head_filerecord=[headfile],
                               budget_filerecord=[budgetfile], printrecord=saverecord)

        obs_data = {
            f'obs_well_{i+1}.csv': [
                (f'well_{i+1}', 'HEAD', (lay, row, col))
            ] for i, (lay, row, col) in enumerate(obs_wells)
        }

        flopy.mf6.ModflowUtlobs(
            gwf,
            digits=10,
            print_input=True,
            continuous=obs_data,
        )

        gwtname = f"gwt_{sim_name}_{run}"
        gwt = flopy.mf6.ModflowGwt(sim, modelname=gwtname, model_nam_file=f"{gwtname}.nam")
        imsgwt = flopy.mf6.ModflowIms(
            sim,
            print_option="SUMMARY",
            outer_dvclose=hclose,
            outer_maximum=nouter,
            under_relaxation="NONE",
            inner_maximum=ninner,
            inner_dvclose=hclose,
            rcloserecord=rclose,
            linear_acceleration="BICGSTAB",
            scaling_method="NONE",
            reordering_method="NONE",
            relaxation_factor=relax,
            filename="{}.ims".format(gwtname),
        )
        sim.register_ims_package(imsgwt, [gwt.name])

        flopy.mf6.ModflowGwtdis(gwt, nlay=nlay, nrow=nrow, ncol=ncol,
                                delr=delr, delc=delc, top=top, botm=botm, idomain=idomain)
        initial_concentration = 0.0
        flopy.mf6.ModflowGwtic(gwt, strt=initial_concentration)
        flopy.mf6.ModflowGwtdsp(gwt, alh=40.0, ath1=4.0, atv=0.1, diffc=0.0)
        flopy.mf6.ModflowGwtmst(gwt, porosity=0.3)
        sourcerecarray = [('WEL-1', 'AUX', 'CONCENTRATION')]
        flopy.mf6.ModflowGwtssm(gwt, sources=sourcerecarray, print_flows=True, filename=f"{gwtname}.ssm")

        gwt_obs = [
            (f"conc_well_{i}", "CONCENTRATION", (lay, row, col))
            for i, (lay, row, col) in enumerate(obs_wells)
        ]
        flopy.mf6.ModflowUtlobs(gwt, pname=f"gwt_{sim_name}_{run}_obs", digits=10, print_input=True, continuous=gwt_obs)

        run_output_dir = os.path.join(output_dir, f"run_{run + 1}")
        os.makedirs(run_output_dir, exist_ok=True)

        concentrationfile = f"{sim_name}_{run}.ucn"
        gwt_budgetfile = f"{sim_name}_{run}.cbc"
        flopy.mf6.ModflowGwtoc(gwt, concentration_filerecord=[concentrationfile],
                               budget_filerecord=[gwt_budgetfile],
                               saverecord=[("CONCENTRATION", "ALL"), ("BUDGET", "ALL")])
        flopy.mf6.ModflowGwfgwt(sim, exgtype='GWF6-GWT6', exgmnamea=gwfname, exgmnameb=gwtname, filename="{}.gwfgwt".format(sim_name))

        sim.write_simulation()
        success, buff = sim.run_simulation()
        if not success:
            raise Exception(f"模型运行失败 for run {run}")